In [8]:
# 필수 라이브러리 설치
!pip install --upgrade datasets transformers fsspec
!pip install torch torchvision torchaudio

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 8.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [torchaudio]3 [torchaudio]]


In [10]:
import torch
print("torch version:", torch.__version__)
print("torch available:", torch.__file__)


torch version: 2.7.0
torch available: /Users/jaewoonamgung/Desktop/Git/kb-tech-creators/.venv/lib/python3.12/site-packages/torch/__init__.py


In [12]:
'''
IMDb 감정 분류(Task-Specific) 실험 코드 / Colab 에서 실행
'''

from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

dataset = load_dataset("imdb", revision="main") # 이진 감정 분류 태스크
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # BERT 토크나이저 (lowercase된 vocab 사용)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

dataset = dataset.map(tokenize, batched=True)
dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# 모델 정의 및 훈련 설정
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=1,
    # 평가 전략 수정
    evaluation_strategy="epoch",
    save_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"].select(range(2000)),
    eval_dataset=dataset["test"].select(range(500))
)

trainer.train()
results = trainer.evaluate()
print("Task-Specific Model Accuracy:", results)

ValueError: PyTorch needs to be installed to be able to return PyTorch tensors.